In [ ]:
! pip install accelerate peft bitsandbytes transformers trl auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os
import json
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:

model_id = "Deci/DeciLM-6b-instruct"

def finetune_deci_lm():
    # data = load_dataset("truongghieu/BK_Regulation_v1",split="train")
    with open('/content/drive/MyDrive/datashet.json', 'r') as file:
      data = json.load(file)
    data_df = pd.DataFrame(data)
    print(data_df.iloc[0])
    data_df["text"] = data_df.apply(lambda x: "###Human: " + str(x["instruct"]) + "\n" + str(x["input"]) + "\n###Assistant: " + str(x["output"]), axis=1)
    print(data_df.iloc[0:2])
    data_df["output"] = data_df["output"].apply(str)
    data = Dataset.from_pandas(data_df)
    tokenizer = AutoTokenizer.from_pretrained(model_id ,  trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    # load model 4bit

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    model.config.use_cache=False
    model.config.pretraining_tp=1

    peft_config = LoraConfig(
        r=32,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    training_arguments = TrainingArguments(
        output_dir="deci-finetuned_BK_Regulation",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=2,
        max_steps=1500,
        fp16=True,
        push_to_hub=True
    )

    trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
    )

    trainer.train()
    trainer.push_to_hub()

if __name__ == "__main__":
    finetune_deci_lm()

instruct       Explain the scope and application of the regul...
input          What is the scope and application of the regul...
output         These regulations pertain to academic affairs ...
instruction                                                  NaN
Name: 0, dtype: object
                                            instruct  \
0  Explain the scope and application of the regul...   
1  Define certain terms used in these regulations...   

                                               input  \
0  What is the scope and application of the regul...   
1  Can you explain the meaning of terms like 'Pro...   

                                              output instruction  \
0  These regulations pertain to academic affairs ...         NaN   
1  In these regulations, a 'Program of study' ref...         NaN   

                                                text  
0  ###Human: Explain the scope and application of...  
1  ###Human: Define certain terms used in these r...  


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/634 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.497300
20,1.972900
30,1.891100
40,1.632100
50,1.561400
60,1.659400
70,1.218500
80,1.419100
90,1.434700
100,1.341300


 Save Model in Google Drive

In [ ]:
# ! cp -r /content/deci-finetuned /content/drive/MyDrive/

**Merge and Push to hub**

In [ ]:
from peft import AutoPeftModelForCausalLM
import torch
import accelerate
import os
model = AutoPeftModelForCausalLM.from_pretrained("truongghieu/deci-finetuned_BK_Regulation", device_map={"":0}, torch_dtype=torch.float16)
model = model.merge_and_unload()
model.push_to_hub("deci-finetuned_BK_Regulation")

adapter_config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

The repository for Deci/DeciLM-6b-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Deci/DeciLM-6b-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


configuration_decilm.py:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Deci/DeciLM-6b-instruct:
- configuration_decilm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/truongghieu/deci-finetuned_BK_Regulation/commit/ab9ee3b7eb1b41839e58ff566b528db57cdcd27a', commit_message='Upload DeciLMForCausalLM', commit_description='', oid='ab9ee3b7eb1b41839e58ff566b528db57cdcd27a', pr_url=None, pr_revision=None, pr_num=None)

# Inference

# Load Model

In [ ]:
! pip install accelerate peft bitsandbytes transformers

NotImplementedError: ignored

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, BitsAndBytesConfig
import torch
import accelerate
model_id = "truongghieu/deci-finetuned_BK_Regulation"

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=True
    )

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.float16 , trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/869 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/51.2M [00:00<?, ?B/s]

In [ ]:
from time import perf_counter
from transformers import GenerationConfig

question = "What are the consequences if a student fails a mandatory course?"

generation_config = GenerationConfig(
    penalty_alpha=0.6,
    do_sample = True,
    top_k=5,
    temperature=0.5,
    repetition_penalty=1.2,
    max_new_tokens=70,
    pad_token_id=tokenizer.eos_token_id
)
start_time = perf_counter()
inputs = tokenizer("###Human:\""+question+"\"\n", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


end_time = perf_counter()
output_time = end_time - start_time
print(f"Time taken for inference: {round(output_time,2)} seconds")

###Human:"What are the consequences if a student fails a mandatory course?"

###Assistant: If a student fails a mandatory course, they must retake it or take an equivalent course to graduate. Failing a required course more than once may result in disqualification from the program. Students failing a course for the third time might be considered for transferring out of the institution. The institution may
Time taken for inference: 11.83 seconds
